In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from auth import *

def KakaoMapAPI(regions):
    # Create an empty DataFrame to store all the restaurant information
    all_df = pd.DataFrame()
    
    # Iterate over each region and search for restaurants using the Kakao Map API
    for region in regions:
        # Set the API endpoint URL
        url = "https://dapi.kakao.com/v2/local/search/keyword.json"
        
        # Set the headers for the API request
        headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
        
        # Set the search parameters, including the radius of the search (20km), the category of the search (food), 
        # and the search query (the region name followed by "restaurant" in Korean)
        params = {
            # "size" : 45,
            "radius" : 20000,
            "category_group_code" : "FD6",
            "query": f"{region} 맛집"
        }
        # params['size'] = 45
        
        # Iterate over each page of search results (up to 45 pages) and append the restaurant information to the DataFrame
        for i in range(1,4):
            # Set the page parameter for the search request
            params['page'] = i
            
            # Send the search request and get the response as JSON
            response = requests.get(url, headers=headers, params=params)
            print(response)
            tmp_json = response.json()['documents']
            
            # Convert the JSON to a DataFrame and append it to the main DataFrame
            df = pd.DataFrame(tmp_json)
            print(set(df['place_name'].to_list()))
            all_df = pd.concat([all_df, df])
    
    # Rename the DataFrame columns to be more descriptive
    all_df = all_df.rename(columns={
        'x': 'diner_lat',
        'y' : 'diner_lon',
        'road_address_name': 'diner_address',
        'place_name' : 'diner_name',
        'category_name' : 'diner_category',
        'phone' : 'diner_phone',
        'id' : 'diner_id'
    })
    
    # Return the final DataFrame with all the restaurant information
    return all_df


In [118]:
tmp_str = '''서초동·잠원동·반포동·방배동·도곡동·양재동·우면동·원지동·염곡동·내곡동·신원동'''

tmp_lst1 = tmp_str.split('·')

## 문자열 전처리

In [116]:
tmp_str = '''명일동, 하일동, 고덕동, 상일동, 길동, 둔촌동, 암사동, 성내동, 풍납동, 천호동, 송파동, 석촌동, 삼전동, 가락동, 이동, 오금동, 방이동, 문정동, 장지동, 거여동, 마천동, 잠실동, 신천동, 일원동, 수서동, 자곡동, 율현동, 세곡동, 역삼동, 포이동, 개포동, 도곡동, 논현동, 신사동, 학동, 압구정동, 청담동, 삼성동, 대치동, 염곡동, 내곡동, 신원동, 양재동, 우면동, 원지동, 잠원동, 서초동, 반포동'''

tmp_lst = tmp_str.split(', ')

In [119]:
tmp_lst+ tmp_lst1

['명일동',
 '하일동',
 '고덕동',
 '상일동',
 '길동',
 '둔촌동',
 '암사동',
 '성내동',
 '풍납동',
 '천호동',
 '송파동',
 '석촌동',
 '삼전동',
 '가락동',
 '이동',
 '오금동',
 '방이동',
 '문정동',
 '장지동',
 '거여동',
 '마천동',
 '잠실동',
 '신천동',
 '일원동',
 '수서동',
 '자곡동',
 '율현동',
 '세곡동',
 '역삼동',
 '포이동',
 '개포동',
 '도곡동',
 '논현동',
 '신사동',
 '학동',
 '압구정동',
 '청담동',
 '삼성동',
 '대치동',
 '염곡동',
 '내곡동',
 '신원동',
 '양재동',
 '우면동',
 '원지동',
 '잠원동',
 '서초동',
 '반포동',
 '서초동',
 '잠원동',
 '반포동',
 '방배동',
 '도곡동',
 '양재동',
 '우면동',
 '원지동',
 '염곡동',
 '내곡동',
 '신원동']

In [108]:
regions = ['광화문', '']
urls_df1 = KakaoMapAPI(tmp_lst)

<Response [200]>
{'이요이요스시 공덕1호점', '영광보쌈', '함루', '뚜띠쿠치나 공덕점', '락희옥 마포본점', '진대감 마포점', '마포왕족발', '파네트', '마포청학동부침개', '마포진짜원조최대포 본점', '마포우사미', '원조마포껍데기집', '진미식당', '황금콩밭', '탑클라우드23'}
<Response [200]>
{'마포유가궁중족발 마포본점', '마포오향족발', '마포소문난원조족발', '롯데시티호텔마포 나루', '호남식당', '원조신촌설렁탕', '우가촌설렁탕', '신성각', '이도맨숀 공덕점', '홍박아구찜', '조개꽃천지 마포공덕점', '곰탕반', '닥터로빈 공덕점', '봉평옹심이메밀칼국수', '엘리스리틀이태리'}
<Response [200]>
{'참복집', '이요이요 파크자이3호점', '스시복', '공덕김밥', '마포나룻가', '공덕닭발왕', '김재운초밥사랑', '호미곶막회물회', '센다이', '장수족발', '버거킹 공덕역점', '라무진 공덕점', '그레이스국밥', '웰빙김치찜', '서리원'}
<Response [200]>
{'이요이요스시 공덕1호점', '롯데시티호텔마포 나루', '아현동간장게장', '함루', '37.5익스프레스 마포공덕점', '김앤김대게 공덕점', '락희옥 마포본점', '홍박아구찜', '파네트', '김재운초밥사랑', '이도맨숀 공덕점', '호미곶막회물회', '황금콩밭', '곱창파는고깃집', '서리원'}
<Response [200]>
{'밀리네해물탕 애오개역점', '25카츠 공덕점', '대가설렁탕', '하남돼지집 애오개역점', '클래식햄버거', '편백집 아현점', '동아냉면 공덕점', '은성순대국', '용가', '소나무숯불구이', '팟타이로얄', '밀밭정원', '황톳길', '서울호떡 아현점', '동신참치 공덕점'}
<Response [200]>
{'뚜레쥬르 공덕역점', '써브웨이 공덕역롯데캐슬점', '칭닌', '선향정샤브샤브', '뺑스톡 공덕점', '메밀뜨락', '맘스터치 마포공덕역점', '더삼겹170', '명

In [109]:
urls_df= pd.concat([urls_df, urls_df1])

In [110]:
urls_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2257 entries, 0 to 14
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   address_name         2257 non-null   object
 1   category_group_code  2257 non-null   object
 2   category_group_name  2257 non-null   object
 3   diner_category       2257 non-null   object
 4   distance             2257 non-null   object
 5   diner_id             2257 non-null   object
 6   diner_phone          2257 non-null   object
 7   diner_name           2257 non-null   object
 8   place_url            2257 non-null   object
 9   diner_address        2257 non-null   object
 10  diner_lat            2257 non-null   object
 11  diner_lon            2257 non-null   object
dtypes: object(12)
memory usage: 229.2+ KB


In [105]:
urls_df

,address_name,category_group_code,category_group_name,diner_category,distance,diner_id,diner_phone,diner_name,place_url,diner_address,diner_lat,diner_lon
0,서울 영등포구 당산동3가 228-1,FD6,음식점,"음식점 > 한식 > 육류,고기",,893093723,010-5852-8749,원조부안집,http://place.map.kakao.com/893093723,서울 영등포구 당산로32길 10,126.89784886434887,37.52623159496171
1,서울 영등포구 당산동1가 79-7,FD6,음식점,"음식점 > 한식 > 해물,생선 > 회",,27287766,02-2068-3430,최우영수산 영등포점,http://place.map.kakao.com/27287766,서울 영등포구 영등포로 161,126.89946569453387,37.52101347686589
2,서울 영등포구 당산동3가 223-3,FD6,음식점,"음식점 > 한식 > 육류,고기",,40064675,,동래정 영등포본점,http://place.map.kakao.com/40064675,서울 영등포구 당산로30길 5,126.897256864032,37.5256508362952
3,서울 영등포구 당산동3가 100,FD6,음식점,음식점 > 한식,,925483333,02-2635-1231,구석집,http://place.map.kakao.com/925483333,서울 영등포구 국회대로36길 3-1,126.898257438195,37.5269338279574
4,서울 영등포구 당산동3가 85-1,FD6,음식점,"음식점 > 한식 > 육류,고기 > 삼겹살",,491305319,0507-1329-1222,냅다청양집 본점,http://place.map.kakao.com/491305319,서울 영등포구 당산로32길 12,126.89813738380003,37.52620211109923
...,...,...,...,...,...,...,...,...,...,...,...,...
10,서울 종로구 명륜4가 154-5,FD6,음식점,음식점 > 양식 > 이탈리안,,27284855,02-741-3288,혜화동다이닝,http://place.map.kakao.com/27284855,서울 종로구 창경궁로26길 37-4,126.99966035909607,37.58206195327818
11,서울 종로구 명륜4가 129,FD6,음식점,음식점 > 일식 > 일본식라면,,2082603117,02-766-6604,부탄츄 대학로4호점,http://place.map.kakao.com/2082603117,서울 종로구 대학로9길 23-2,127.00045285088213,37.5814582871732
12,서울 종로구 명륜4가 66-2,FD6,음식점,음식점 > 중식 > 중국요리,,13627376,02-765-5688,진아춘,http://place.map.kakao.com/13627376,서울 종로구 대명1길 18,127.00053324177793,37.5828413119516
13,서울 종로구 명륜2가 27-1,FD6,음식점,음식점 > 술집 > 실내포장마차,,1570875666,,혜화시장,http://place.map.kakao.com/1570875666,서울 종로구 창경궁로34길 24-6,127.00083327368064,37.58397566086413


In [37]:
urls_df.head()

,address_name,category_group_code,category_group_name,diner_category,distance,diner_id,diner_phone,diner_name,place_url,diner_address,diner_lat,diner_lon
0,서울 강남구 삼성동 148-15,FD6,음식점,음식점 > 중식,,1770731230,0507-1374-3082,팀호완 삼성점,http://place.map.kakao.com/1770731230,서울 강남구 봉은사로86길 30,127.056847505366,37.5114270736423
1,서울 강남구 삼성동 113,FD6,음식점,음식점 > 한식 > 한정식,,25744174,02-558-4900,수담한정식,http://place.map.kakao.com/25744174,서울 강남구 봉은사로 418,127.045938976373,37.510603796848
2,서울 강남구 삼성동 151-7,FD6,음식점,음식점 > 한식 > 냉면,,27261403,02-569-8939,능라도 강남점,http://place.map.kakao.com/27261403,서울 강남구 삼성로 534,127.05553416938561,37.509677046854954
3,서울 강남구 삼성동 58-6,FD6,음식점,"음식점 > 한식 > 육류,고기",,27512695,02-518-9710,길목,http://place.map.kakao.com/27512695,서울 강남구 영동대로129길 10,127.0564630117036,37.519026323125196
4,서울 강남구 삼성동 142-46,FD6,음식점,"음식점 > 한식 > 육류,고기",,21306153,02-553-2237,경천애인2237 삼성본점,http://place.map.kakao.com/21306153,서울 강남구 테헤란로 421,127.05255692332769,37.50596086312871


In [111]:
len(set(urls_df['place_url'].to_list()))

2237

In [95]:
urls_df.iloc[:,[7,8]]

,diner_name,place_url
0,원조부안집,http://place.map.kakao.com/893093723
1,최우영수산 영등포점,http://place.map.kakao.com/27287766
2,동래정 영등포본점,http://place.map.kakao.com/40064675
3,구석집,http://place.map.kakao.com/925483333
4,냅다청양집 본점,http://place.map.kakao.com/491305319
...,...,...
10,하우돈곱창 영등포점,http://place.map.kakao.com/17198092
11,한옥집김치찜 영등포점,http://place.map.kakao.com/844765261
12,이가생고기참숯불갈비,http://place.map.kakao.com/16331633
13,얌샘김밥 플러스 영등포역점,http://place.map.kakao.com/14084044


## 중복 파악

In [112]:
urls_df.drop_duplicates(subset=['place_url','diner_name'], keep='last',inplace=True)

In [114]:
urls_df.to_csv('KakaoApi_tmp.csv', index=False, encoding='utf-8-sig')

# 리뷰 긁기

In [115]:
import os
import re
from time import sleep
import requests

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import (
    ElementNotInteractableException,
    NoSuchElementException,
    StaleElementReferenceException,
)
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("lang=ko_KR")
options.add_argument("headless")
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")

# chromedriver_path = "/home/elinha/Testproject/mediapipe/chromedriver"
# 크롬 드라이버를 사용합니다 (맥은 첫 줄, 윈도우는 두번째 줄 실행)
ex_path =ChromeDriverManager().install()


In [15]:
diner_cols = [
        'diner_id',
        'diner_name',           # 가게이름 
        'diner_category',       # 가게 카테고리
        'diner_menu',           # 가게 메뉴
        'diner_review_cnt',     # 가게의 평점 개수 
        'diner_review_avg',     # 가게의 평점 평균
        'diner_review_tags',    # 리뷰 태그
        'diner_address',        # 가게 주소    
        'diner_lon',            # 가게 위도
        'diner_lat',            # 가게 경도
        'diner_url',            # 가게 URL
        'diner_open_time'       # 가게 오픈시간
       ]

review_cols = [
        'diner_id',
        'reviewer_review',
        'reviewer_avg',         # 리뷰어의 평점 평균
        'reviewer_review_cnt',  # 리뷰어의 리뷰 개수
        'reviewer_review_score',# 리뷰어가 남긴 평점
        'reviewer_review_date', # 리뷰를 남긴 날짜
        'reviewer_id'
        ]

def MakeText(lst):
    return [i.text for i in lst]

# 사업장명, 주소, 음식종류1,음식종류2(메뉴),리뷰수,별점,리뷰
dinner_df = pd.DataFrame(columns=diner_cols)
review_df = pd.DataFrame(columns=review_cols)
driver = webdriver.Chrome(executable_path=ex_path, options=options)
for i in range(len(urls_df)):
    # if i == 1:
    #     break
    print(i)
    diner_id = urls_df.iloc[i,5]
    page_url = urls_df.iloc[i,8]
    cat1 = urls_df.iloc[i,3]
    address = urls_df.iloc[i,9]
    name  = urls_df.iloc[i,7]
    diner_phone = urls_df.iloc[i,6]
    diner_lat = urls_df.iloc[i,10]
    diner_lon = urls_df.iloc[i,11]
    print(f"{diner_id}: {page_url}")
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'kakaoWrap')))
    sleep(3)


    if driver.find_elements(
        By.XPATH, '//div[@class="box_grade_off"]'
    ):  # 후기를 제공하지 않는 맛집 넘기기
        continue
    
    # 평균 별점
    score_min = driver.find_element(
        By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[1]/span[1]'
    ).text

    # 리뷰수
    review_num = driver.find_element(By.XPATH, '//span[@class="color_g"]').text[1:-2]
    
    #영업시간
    open_times = driver.find_elements(By.XPATH, '//ul[@class="list_operation"]')
    
    if len(open_times):
        open_time = open_times[0].get_attribute('innerText').split('\n')[0]
    else:
        open_time = '제공하지 않음'

    # # 블로그리뷰수
    # blog_review_num = driver.find_element(
    #     By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/div/a[2]/span'
    # ).text


    # 메뉴
    cat2 = []

    menus = driver.find_elements(By.CLASS_NAME, "info_menu")
    for menu in menus:
        cat2.append(menu.text)

    # 식당 장점
    likePoints = driver.find_elements(By.XPATH, '//*[@class="txt_likepoint"]')
    likePointCnts = driver.find_elements(By.XPATH, '//*[@class="num_likepoint"]')
    likePoint = ""
    for p, c in zip(likePoints, likePointCnts):
        likePoint += p.text + "@" + c.text + "@"

    if driver.find_elements(By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a/span[1]'):
        # 리뷰 더보기 최대로
        while not bool(
            driver.find_elements(By.XPATH, '//a[@class="link_more link_unfold"]')
        ):
            tmp_clk = driver.find_elements(By.XPATH, '//span[@class="txt_more"]')
            wait = WebDriverWait(driver, 1)
            element = wait.until(
                EC.element_to_be_clickable((By.CLASS_NAME, "link_more"))
            )
            try:
                if tmp_clk[0].text == "후기 더보기":
                    tmp_clk[0].click()
            except Exception as e:
                print("클릭 예외가 발생되었습니다.")
                pass

    try:
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        contents_div = soup.find(name="div", attrs={"class": "evaluation_review"})

        # 별점을 가져옵니다.
        rateNcnt = contents_div.find_all(name="span", attrs={"class": "txt_desc"})
        rateCnts = rateNcnt[::2]
        rates = rateNcnt[1::2]

        # 개인이 해당 식당에 남긴 별점
        rateAts_tmp = driver.find_elements(
            By.XPATH, '//div[@class="grade_star size_s"]/span/span'
        )

        # 리뷰를 가져옵니다.
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"})

        # 리뷰를 쓴 날짜를 가져옵니다.
        reviews_dates = contents_div.find_all(
            name="span", attrs={"class": "time_write"}
        )

        # 리뷰 아이디 가져오기
        reviews_ids = contents_div.find_all(name="a", attrs={"class": "link_user"})
        

        dinner_row = [
                diner_id,
                name,
                cat1,
                cat2,
                review_num,
                score_min,
                likePoint,
                address,
                diner_lon,
                diner_lat,
                page_url,
                open_time,
            ]
        series = pd.DataFrame([dinner_row], columns=diner_cols)
        dinner_df = pd.concat([dinner_df, series])

        rateAts = [int(rateAt.get_attribute("style")[7:-2]) / 20 for rateAt in rateAts_tmp]
        reviews = [review.find(name="span").text for review in reviews]
        rates = MakeText(rates)
        rateCnts = MakeText(rateCnts)
        reviews_dates = MakeText(reviews_dates)
        reviews_ids = MakeText(reviews_ids)

        print("rateAts", len(rateAts), "reviews", len(reviews))

        # Create a dictionary with the data
        review_data = {
        'reviewer_review': reviews,
        'reviewer_avg': rates[:len(reviews)],
        'reviewer_review_cnt': rateCnts[:len(reviews)],
        'reviewer_review_score': rateAts[:len(reviews)],
        'reviewer_review_date': reviews_dates[:len(reviews)],
        'reviewer_id': reviews_ids[:len(reviews)],
        }
        
        review_df_tmp = pd.DataFrame(review_data)
        review_df_tmp['diner_id'] = diner_id
        review_df = pd.concat([review_df, review_df_tmp])
    except Exception as e:
        print("예외가 발생되었습니다.", e)
review_df.drop_duplicates(subset=['reviewer_id','reviewer_review','reviewer_review_cnt','reviewer_review_score','reviewer_review_date'], keep='last',inplace=True)

/tmp/ipykernel_331088/2769496600.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ex_path, options=options)


1770731230: http://place.map.kakao.com/1770731230
클릭 예외가 발생되었습니다.
rateAts 490 reviews 490
25744174: http://place.map.kakao.com/25744174
rateAts 3 reviews 3
27261403: http://place.map.kakao.com/27261403
rateAts 3 reviews 3
27512695: http://place.map.kakao.com/27512695
클릭 예외가 발생되었습니다.
rateAts 464 reviews 464
21306153: http://place.map.kakao.com/21306153
클릭 예외가 발생되었습니다.
rateAts 465 reviews 465
8664636: http://place.map.kakao.com/8664636
rateAts 3 reviews 3
1198484281: http://place.map.kakao.com/1198484281
클릭 예외가 발생되었습니다.
rateAts 371 reviews 371
26794399: http://place.map.kakao.com/26794399
클릭 예외가 발생되었습니다.
클릭 예외가 발생되었습니다.
rateAts 289 reviews 289
8024422: http://place.map.kakao.com/8024422
클릭 예외가 발생되었습니다.
rateAts 326 reviews 326
12890589: http://place.map.kakao.com/12890589
클릭 예외가 발생되었습니다.
rateAts 258 reviews 258
24041168: http://place.map.kakao.com/24041168
클릭 예외가 발생되었습니다.
rateAts 96 reviews 96
8969599: http://place.map.kakao.com/8969599
rateAts 107 reviews 107
1986314222: http://place.map

In [158]:
review_df_tmp['diner_id'] = diner_id

In [18]:
dinner_df

,diner_id,diner_name,diner_category,diner_menu,diner_review_cnt,diner_review_avg,diner_review_tags,diner_address,diner_lon,diner_lat,diner_url,diner_open_time
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,25744174,수담한정식,음식점 > 한식 > 한정식,"[보리굴비정식\n28,000\n점심, 선비상\n38,000, 대감상\n48,000,...",4,3.5,,서울 강남구 봉은사로 418,37.510603796848,127.045938976373,http://place.map.kakao.com/25744174,매일 11:30 ~ 22:00
0,27261403,능라도 강남점,음식점 > 한식 > 냉면,"[평양냉면\n15,000, 비빔면\n15,000, 평양온면\n15,000, 평양온반...",11,3.7,맛@11@친절@5@분위기@5@주차@3@가성비@2@,서울 강남구 삼성로 534,37.509677046854954,127.05553416938561,http://place.map.kakao.com/27261403,매일 11:20 ~ 21:30
0,27512695,길목,"음식점 > 한식 > 육류,고기","[투뿔목살(200g)\n18,000, 특오겹살(200g)\n18,000, 껍살(25...",9,4.2,맛@14@친절@9@가성비@3@분위기@2@주차@1@,서울 강남구 영동대로129길 10,37.519026323125196,127.0564630117036,http://place.map.kakao.com/27512695,월~토 16:00 ~ 23:00
0,21306153,경천애인2237 삼성본점,"음식점 > 한식 > 육류,고기","[경애등.안.채 (150g)\n45,500, 특안심 (150g)\n49,500, 꽃...",8,4.1,맛@9@친절@6@가성비@4@분위기@4@,서울 강남구 테헤란로 421,37.50596086312871,127.05255692332769,http://place.map.kakao.com/21306153,월~토 11:30 ~ 21:00
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1625016339,무초,"음식점 > 한식 > 육류,고기","[드라이에이징 티본 (100g)\n22,000, 드라이에이징 채끝(150g)\n24...",1,3.9,,서울 서초구 마방로10길 18-16,37.476078264729495,127.04312993413537,http://place.map.kakao.com/1625016339,월~금 11:30 ~ 21:30
0,16055366,박고볼래,음식점 > 술집 > 실내포장마차,"[물회\n28,000, 멍게\n18,000, 뼈없는 닭발\n20,000, 육전\n2...",1,3.4,맛@2@분위기@1@,서울 서초구 강남대로30길 12,37.4817491988487,127.037930075617,http://place.map.kakao.com/16055366,매일 17:00 ~ 04:00
0,19207735,마오 양재점,음식점 > 중식 > 중국요리,"[자장면\n6,000, 새우볶음밥\n8,000, 마라샹궈\n38,000, 마라탕 (...",4,2.6,맛@1@가성비@1@분위기@1@,서울 서초구 양재천로 105,37.4767928412744,127.040365028445,http://place.map.kakao.com/19207735,매일 11:30 ~ 01:00
0,379369560,정석초밥,"음식점 > 일식 > 초밥,롤","[광어초밥 (1pcs)\n2,000, 연어초밥 (1pcs)\n1,500, 계란초밥 ...",3,3.4,,서울 서초구 강남대로12길 41,37.4707621616585,127.041001623161,http://place.map.kakao.com/379369560,월~토 11:00 ~ 22:00 (재료소진시 마감)


In [30]:
NameCheck_lst = []
for name in list(set(review_df['diner_id'].to_list())):
    print(name)
    NameCheck_lst.append(dinner_df[dinner_df['diner_id'] == name].iloc[0,1])

24075197
19514179
137659890
1770731230
7881618
8214517
917389351
1123837399
27422536
8664636
16649489
1532194987
1499028769
1498098673
1380126539
10825881
21554473
368469701
1681932014
954098848
19207735
7865634
10559917
21960500
27367068
640778161
13291936
27512695
8969599
948434557
1326153988
18128127
1193414362
337259331
8024422
857407201
27285836
27274125
26534500
1625016339
830612493
1485180110
27261403
21306153
26410902
704203309
27388660
1986314222
8701738
379369560
659394180
1198484281
16055366
26428654
779959518
7974994
25744174
17462492
7885177
900538186
96772273
96268218
14480921
18377454
9063870
14872146
27271692
23985120
17363119
1232624597
8117481
12701868
22757053
12890589
24041168
19305786
1923465263
21387721
12428506
23985599
119019242
7850610
16055426
11831453
8686213
1777207600
26794399
2117464704
1211307526


In [36]:
len(set(dinner_df['diner_name'].to_list()))

89

In [33]:
for name in list(set(dinner_df['diner_name'].to_list())):
    print(name)

박서방순대국밥 삼성본점
탐라도야지
영동족발 본점
마노디셰프 삼성점
대도식당 삼성점
그랜드인터컨티넨탈 서울파르나스 그랜드키친
교대이층집 양재점
곰바위
차이린
브루클린더버거조인트 삼성점
하피덕
이남장 삼성점
영동족발 3호점
풍천가 서초점
오선채
미우야
소백산 양재별관
한국순대 본점
박고볼래
시추안하우스 삼성점
봉밀가
브루스리
이대성의 진면목
식감이야기 양재역점
인터컨티넨탈서울코엑스 브래서리
무초
세광양대창 양재점
배나무골 양재본점
수다 삼성 2호점
그릴1492 삼성1호점
장정정
길목
디오디아2.0
소호정 본점
돼지대첩
고메램 삼성점
치코
능라도 강남점
외고집설렁탕
흑돈가 삼성역점
마포네숯불돼지갈비
조인바이트
아랑솥뚜껑 양재본점
정석초밥
일일향 삼성점
김돈이 본점
이가네양꼬치 양재2호점
마오 양재점
일품헌
비야게레로
경천애인2237 삼성본점
두어마리 양재점
인덕원비빔국수
강촌원조쭈꾸미 본점
하동관 코엑스몰직영점
백세주마을 삼성점
사람사는고깃집 김일도 삼성코엑스점
문어세상 양재점
붓처스컷 삼성점
바이킹스워프 코엑스점
옛날농장 매헌점
수담한정식
삼성리
도토리편백집 양재시민의숲점
라비드쿤
메르시
한우라서더맛있는스테이크
서울양고기
더차이홍 삼성점
산들해 양재동점
스시히로바 삼성동본점
백화네부엌
더라운드 삼성점
바베큐랩
젠제로
호신각
갈비사랑
아우정
양재닭집
수다 삼성1호점
평가옥 삼성점
디 안다만
구을가
강남한우정육식당
설마중
명인등심 삼성직영점
잇쇼우
해물과차돌박이 양재점
팀호완 삼성점


In [35]:
set(dinner_df['diner_id'].to_list()) - set(review_df['diner_id'].to_list())

set()

In [31]:
NameCheck_lst

['한우라서더맛있는스테이크',
 '붓처스컷 삼성점',
 '백화네부엌',
 '팀호완 삼성점',
 '박서방순대국밥 삼성본점',
 '그랜드인터컨티넨탈 서울파르나스 그랜드키친',
 '라비드쿤',
 '이가네양꼬치 양재2호점',
 '문어세상 양재점',
 '곰바위',
 '이남장 삼성점',
 '사람사는고깃집 김일도 삼성코엑스점',
 '돼지대첩',
 '더라운드 삼성점',
 '풍천가 서초점',
 '인터컨티넨탈서울코엑스 브래서리',
 '디 안다만',
 '조인바이트',
 '잇쇼우',
 '일일향 삼성점',
 '마오 양재점',
 '마포네숯불돼지갈비',
 '소호정 본점',
 '일품헌',
 '두어마리 양재점',
 '고메램 삼성점',
 '강촌원조쭈꾸미 본점',
 '길목',
 '마노디셰프 삼성점',
 '호신각',
 '세광양대창 양재점',
 '옛날농장 매헌점',
 '도토리편백집 양재시민의숲점',
 '장정정',
 '외고집설렁탕',
 '교대이층집 양재점',
 '탐라도야지',
 '명인등심 삼성직영점',
 '서울양고기',
 '무초',
 '아우정',
 '더차이홍 삼성점',
 '능라도 강남점',
 '경천애인2237 삼성본점',
 '설마중',
 '해물과차돌박이 양재점',
 '대도식당 삼성점',
 '바이킹스워프 코엑스점',
 '갈비사랑',
 '정석초밥',
 '하피덕',
 '이대성의 진면목',
 '박고볼래',
 '하동관 코엑스몰직영점',
 '그릴1492 삼성1호점',
 '스시히로바 삼성동본점',
 '수담한정식',
 '미우야',
 '백세주마을 삼성점',
 '젠제로',
 '식감이야기 양재역점',
 '오선채',
 '수다 삼성1호점',
 '브루클린더버거조인트 삼성점',
 '산들해 양재동점',
 '배나무골 양재본점',
 '비야게레로',
 '강남한우정육식당',
 '한국순대 본점',
 '인덕원비빔국수',
 '평가옥 삼성점',
 '차이린',
 '아랑솥뚜껑 양재본점',
 '김돈이 본점',
 '시추안하우스 삼성점',
 '치코',
 '구을가',
 '소백산 양재별관',
 '양재닭집',
 '영동족발 본점',
 '삼성리',
 '디오디

In [19]:
review_df_tmp

,reviewer_review,reviewer_avg,reviewer_review_cnt,reviewer_review_score,reviewer_review_date,reviewer_id,diner_id
0,어머님도 친절하시고 맛도 있고,4.5,15,5.0,2021.12.22.,KK,27367068
1,,3.5,74,2.0,2020.05.27.,오,27367068
2,,3.6,28,4.0,2020.05.22.,ㅎㅎ,27367068


In [153]:
len(review_df['reviewer_review'].to_list())

194

In [96]:
review_df.drop_duplicates(subset=['reviewer_id','reviewer_review','reviewer_review_cnt','reviewer_review_score','reviewer_review_date'], keep='last',inplace=True)

In [ ]:
len()

In [102]:
review_df[review_df['diner_id'] == "12890589"]

,diner_id,reviewer_review,reviewer_avg,reviewer_review_cnt,reviewer_review_score,reviewer_review_date,reviewer_id
0,12890589,제주 흙돼지 고기집이 흔해져 이젠 강점이 사라진 느낌. 점심 메뉴는 취약하고 저녁 ...,3.7,175,3.0,2023.03.24.,2Roon
0,12890589,여기 수준급의 근고기집이었는데 작년인가부터 맛이 변해서 이제는 그저그런 고깃집ㅜ 일...,3.1,99,3.0,2023.02.16.,대치동요리왕
0,12890589,흠...,3.2,702,1.0,2022.11.25.,음음
0,12890589,21.07.07 ~ 21.07.155점리뷰 알바들의 흔적이 보인다거짓 리뷰를 쓰게 ...,2.6,101,1.0,2022.11.15.,리뷰알바검거.👍=끌올
0,12890589,외국인 친구랑 같이 갔는데 친절하게 대해주시고 말도 걸어주시고 좋았어요! 두번 갔는...,3.9,247,4.0,2022.10.17.,맛있는걸 조지는 조지나
...,...,...,...,...,...,...,...
0,12890589,,3.8,39,5.0,2016.08.18.,박영준_Dyne
0,12890589,두툼한 돼지고기,4.1,231,4.0,2016.07.23.,윤현준
0,12890589,,4.1,128,4.0,2016.06.30.,어라?
0,12890589,제주도 흑돼지를 서울에서도 맛볼 수 있다!!!몇일전 제주도에서 먹은 맛이다!!!사람...,3.9,180,5.0,2016.04.24.,별별


In [88]:
!pip install openpyxl


In [46]:
diner_cols = [
    'dinner_id',
    'diner_name', # restaurant name
    'diner_category', # shop category
    'diner_menu', # shop menu
    'diner_review_cnt', # the number of reviews in the store
    'diner_review_avg', # Average rating for the store
    'diner_review_tags', # review tags
    'diner_address', # store address
    'diner_lon',
    'diner_lat',
    'diner_url',
    'diner_open_time' # store open time
]

diner_df = df[diner_cols]

review_cols = [

    'reviewer_review',
    'reviewer_avg', # the reviewer's rating average
    'reviewer_review_cnt', # number of reviews by reviewer
    'reviewer_review_score',# the rating left by the reviewer
    'reviewer_review_date', # Date the review was left
]

review_df = df[review_cols]

In [21]:
 # write each dataframe to a separate sheet in the Excel file
dinner_df.to_csv('dinner_test.csv', index=False, encoding='utf-8-sig')
review_df.to_csv('review_test.csv', index=False, encoding='utf-8-sig')

In [67]:
diner_df

,dinner_id,diner_name,diner_category,diner_menu,diner_review_cnt,diner_review_avg,diner_review_tags,diner_address,diner_lon,diner_lat,diner_url,diner_open_time
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
...,...,...,...,...,...,...,...,...,...,...,...,...
0,19207735,마오 양재점,음식점 > 중식 > 중국요리,"[자장면\n6,000, 새우볶음밥\n8,000, 마라샹궈\n38,000, 마라탕 (...",4,2.6,맛@1@가성비@1@분위기@1@,서울 서초구 양재천로 105,37.4767928412744,127.040365028445,http://place.map.kakao.com/19207735,매일 11:30 ~ 01:00
0,19207735,마오 양재점,음식점 > 중식 > 중국요리,"[자장면\n6,000, 새우볶음밥\n8,000, 마라샹궈\n38,000, 마라탕 (...",4,2.6,맛@1@가성비@1@분위기@1@,서울 서초구 양재천로 105,37.4767928412744,127.040365028445,http://place.map.kakao.com/19207735,매일 11:30 ~ 01:00
0,27367068,두어마리 양재점,"음식점 > 한식 > 해물,생선 > 장어",[],1,3.6,,서울 서초구 양재천로23길 6,37.47808820158043,127.04120912730333,http://place.map.kakao.com/27367068,일요일
0,27367068,두어마리 양재점,"음식점 > 한식 > 해물,생선 > 장어",[],1,3.6,,서울 서초구 양재천로23길 6,37.47808820158043,127.04120912730333,http://place.map.kakao.com/27367068,일요일


In [68]:
diner_df.drop_duplicates(subset=['dinner_id'], keep='last',inplace=True)

<ipython-input-68-92a92b1c67b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diner_df.drop_duplicates(subset=['dinner_id'], keep='last',inplace=True)


In [69]:
diner_df

,dinner_id,diner_name,diner_category,diner_menu,diner_review_cnt,diner_review_avg,diner_review_tags,diner_address,diner_lon,diner_lat,diner_url,diner_open_time
0,1770731230,팀호완 삼성점,음식점 > 중식,"[샤오마이 (3pcs)\n6,500, 차슈 라이스 롤 (3pcs)\n8,000, 사...",19,3.5,맛@9@분위기@4@친절@3@주차@3@가성비@2@,서울 강남구 봉은사로86길 30,37.5114270736423,127.056847505366,http://place.map.kakao.com/1770731230,매일 11:30 ~ 21:30
0,25744174,수담한정식,음식점 > 한식 > 한정식,"[보리굴비정식\n28,000\n점심, 선비상\n38,000, 대감상\n48,000,...",4,3.5,,서울 강남구 봉은사로 418,37.510603796848,127.045938976373,http://place.map.kakao.com/25744174,매일 11:30 ~ 22:00
0,27261403,능라도 강남점,음식점 > 한식 > 냉면,"[평양냉면\n15,000, 비빔면\n15,000, 평양온면\n15,000, 평양온반...",11,3.7,맛@11@친절@5@분위기@5@주차@3@가성비@2@,서울 강남구 삼성로 534,37.509677046854954,127.05553416938561,http://place.map.kakao.com/27261403,매일 11:20 ~ 21:30
0,27512695,길목,"음식점 > 한식 > 육류,고기","[투뿔목살(200g)\n18,000, 특오겹살(200g)\n18,000, 껍살(25...",9,4.2,맛@14@친절@9@가성비@3@분위기@2@주차@1@,서울 강남구 영동대로129길 10,37.519026323125196,127.0564630117036,http://place.map.kakao.com/27512695,일요일
0,21306153,경천애인2237 삼성본점,"음식점 > 한식 > 육류,고기","[경애등.안.채 (150g)\n45,500, 특안심 (150g)\n49,500, 꽃...",8,4.1,맛@9@친절@6@가성비@4@분위기@4@,서울 강남구 테헤란로 421,37.50596086312871,127.05255692332769,http://place.map.kakao.com/21306153,일요일
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1625016339,무초,"음식점 > 한식 > 육류,고기","[드라이에이징 티본 (100g)\n22,000, 드라이에이징 채끝(150g)\n24...",1,3.9,,서울 서초구 마방로10길 18-16,37.476078264729495,127.04312993413537,http://place.map.kakao.com/1625016339,일요일
0,16055366,박고볼래,음식점 > 술집 > 실내포장마차,"[물회\n28,000, 멍게\n18,000, 뼈없는 닭발\n20,000, 육전\n2...",1,3.4,맛@2@분위기@1@,서울 서초구 강남대로30길 12,37.4817491988487,127.037930075617,http://place.map.kakao.com/16055366,매일 17:00 ~ 04:00
0,379369560,정석초밥,"음식점 > 일식 > 초밥,롤","[광어초밥 (1pcs)\n2,000, 연어초밥 (1pcs)\n1,500, 계란초밥 ...",3,3.4,,서울 서초구 강남대로12길 41,37.4707621616585,127.041001623161,http://place.map.kakao.com/379369560,일요일
0,19207735,마오 양재점,음식점 > 중식 > 중국요리,"[자장면\n6,000, 새우볶음밥\n8,000, 마라샹궈\n38,000, 마라탕 (...",4,2.6,맛@1@가성비@1@분위기@1@,서울 서초구 양재천로 105,37.4767928412744,127.040365028445,http://place.map.kakao.com/19207735,매일 11:30 ~ 01:00


In [64]:
review_df.drop_duplicates(subset=['reviewer_id','reviewer_review','reviewer_review_cnt','reviewer_review_score','reviewer_review_date'], keep='last',inplace=True)

In [65]:
review_df

,diner_id,reviewer_review,reviewer_avg,reviewer_review_cnt,reviewer_review_score,reviewer_review_date,reviewer_id
0,1770731230,가격이 좀 나가고딤섬이 3개라 좀 아쉽구요.발렛비가 4천원이라 세요,3.3,155,3.0,2023.04.02.,Jin Legend
0,1770731230,맛이 너무 변했어요,3.3,55,1.0,2023.03.30.,미식가
0,1770731230,싼 가격에 여러가지 음식을 먹어볼수 있는데 맛이 엄청 맛있다고하는 건 없음,4.2,297,3.0,2023.03.28.,맛집원정대따봉도치
0,1770731230,jmt,3.5,27,5.0,2023.03.26.,ㅇ
0,1770731230,다양한 메뉴 모두 기대되는 맛.,3.7,612,4.0,2023.03.22.,바람식객
...,...,...,...,...,...,...,...
0,19207735,엥 여기 맛있는데 메뉴 시킨거 맛 빠지는거 하나 없었어요..근데 평이 안좋네요 흑흑,3.0,3,4.0,2023.03.08.,퍼비
0,19207735,발렛 직원이 건달인가요?,3.1,20,2.0,2022.09.28.,싼티니스타
0,27367068,어머님도 친절하시고 맛도 있고,4.5,15,5.0,2021.12.22.,KK
0,27367068,,3.5,74,2.0,2020.05.27.,오


In [63]:
len(set(review_df['reviewer_review'].tolist()))

2516

In [44]:
df.to_csv('WhatToEat_DB_raw.csv', index=False, encoding='utf-8-sig')